In [1]:
import os
import pandas as pd
import scanpy as sc
os.chdir(path='../')
import STForte.helper as stfhelper
from STForte import STGraph
from STForte import STForteModel
from STForte.helper import save_gdata
trial_name = "trial-mouse_brain_coronal_Xenium"

/home/poncey/miniconda3/envs/stforte_env/lib/python3.9/site-packages/pytorch_lightning/utilities/seed.py:47: LightningDeprecationWarning: `pytorch_lightning.utilities.seed.seed_everything` has been deprecated in v1.8.0 and will be removed in v2.0.0. Please use `lightning_fabric.utilities.seed.seed_everything` instead.
  rank_zero_deprecation(
Global seed set to 0


In [2]:
adata = sc.read_10x_h5(
    filename="../stforte/data/xenium/Xenium_V1_FF_Mouse_Brain_Coronal_feature_matrix.h5"
)
df = pd.read_csv(
    "../stforte/data/xenium/Xenium_V1_FF_Mouse_Brain_Coronal_outscells.csv"
)
df.set_index(adata.obs_names, inplace=True)
adata.obs = df.copy()
adata.obsm["spatial"] = adata.obs[["x_centroid", "y_centroid"]].copy().to_numpy()
adata

AnnData object with n_obs × n_vars = 130870 × 248
    obs: 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
    var: 'gene_ids', 'feature_types', 'genome'
    obsm: 'spatial'

In [3]:
# QC
sc.pp.filter_cells(adata, min_counts=10)
sc.pp.filter_genes(adata, min_cells=5)
# Normalization
adata.layers['count'] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata

AnnData object with n_obs × n_vars = 130557 × 248
    obs: 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'n_counts'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'
    uns: 'log1p'
    obsm: 'spatial'
    layers: 'count'

In [4]:
stgraph = STGraph.graphFromAnndata(adata=adata, 
                                   knn=True,  # Directly initializing knn for non-lattice data.
                                   kdtree=True,
                                   k=10)
gdata = stgraph.topyg()
print(f"The Data contians {gdata.x.size(0)} voxels.")

KDTree knn innitialize.
FP
The Data contians 130557 voxels.


In [5]:
model = STForteModel(adata=adata, gdata=gdata,
                     epochs=500,
                     output_dir='./{:s}/pl_ckpts/'.format(trial_name),
                     module_kwargs=dict(partial_adjacent=True))
model

In [6]:
model.fit()

Global seed set to 42
/home/poncey/miniconda3/envs/stforte_env/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:474: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: trial-mouse_brain_coronal_Xenium_raw/pl_ckpts/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name          | Type              | Params
----------------------------------------------------
0 | attr_encoder  | Sequential        | 61.5 K
1 | strc_encoder  | Sequential_b837d5 | 61.5 K
2 | attr_decoder  | Sequential        | 61.7 K
3 | strc_decoder  | Sequential        | 2.1 K 
4 | discriminator | Sequential        | 1.1 K 
-----------------------------------

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=500` reached.


In [7]:
if not os.path.exists(f"{trial_name}/data"):
    os.makedirs(f"{trial_name}/data")
save_gdata(gdata, path=f"{trial_name}/data/gdata.pkl")
adata.write_h5ad(f"{trial_name}/data/trial.h5ad")
model.save_state_dict(f'./{trial_name}/models/', alias="STForteModel_mouse_brain_xenium")

In [8]:
# For analysis in original resolution
model.get_latent_original(adata)
if not os.path.exists(f"./{trial_name}/outputs"):
    os.makedirs(f"./{trial_name}/outputs")
adata.write(f"./{trial_name}/outputs/stforte.h5ad")